In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
train = pd.read_csv('Training.csv')
train.head()

,Unnamed: 0,Financial Year,Quarter Ending,Quarter,County,City,County_City,key,GDP
0,0,2012,09/30/2011,1,1,1,1_1,93020111,3449658.32
1,1,2012,12/31/2011,2,1,1,1_1,123120111,3433050.00
2,2,2012,03/31/2012,3,1,1,1_1,33120121,3520501.00
3,3,2012,06/30/2012,4,1,1,1_1,63020121,2268743.00
4,4,2013,09/30/2012,1,1,1,1_1,93020121,3515263.00


In [3]:
train.shape

(33734, 9)

In [4]:
train.isnull().sum()

Unnamed: 0        0
Financial Year    0
Quarter Ending    0
Quarter           0
County            0
City              0
County_City       0
key               0
GDP               0
dtype: int64

In [5]:
train['County'].nunique()

99

In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33734 entries, 0 to 33733
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      33734 non-null  int64  
 1   Financial Year  33734 non-null  int64  
 2   Quarter Ending  33734 non-null  object 
 3   Quarter         33734 non-null  int64  
 4   County          33734 non-null  int64  
 5   City            33734 non-null  int64  
 6   County_City     33734 non-null  object 
 7   key             33734 non-null  int64  
 8   GDP             33734 non-null  float64
dtypes: float64(1), int64(6), object(2)
memory usage: 2.3+ MB


In [7]:
train['County_City'].nunique()

775

In [8]:
train['County_City'].unique()

array(['1_1', '1_2', '1_4', '1_5', '1_6', '1_7', '1_8', '2_9', '2_7',
       '3_13', '3_14', '3_15', '3_7', '3_16', '3_17', '3_18', '4_19',
       '4_20', '4_22', '4_23', '4_7', '5_28', '5_30', '5_7', '6_32',
       '6_33', '6_34', '6_35', '6_36', '6_39', '6_40', '6_7', '6_41',
       '6_42', '6_43', '6_44', '6_45', '7_46', '7_47', '7_48', '7_49',
       '7_50', '7_51', '7_52', '7_54', '7_7', '7_55', '7_56', '8_58',
       '8_59', '8_60', '8_7', '9_62', '9_52', '9_7', '9_64', '9_65',
       '9_66', '9_67', '9_68', '10_69', '10_70', '10_71', '10_72',
       '10_73', '10_53', '10_74', '10_7', '10_75', '10_76', '10_78',
       '11_79', '11_80', '11_82', '11_83', '11_84', '11_7', '11_86',
       '11_87', '12_88', '12_89', '12_91', '12_92', '12_93', '12_94',
       '12_7', '12_95', '12_96', '13_97', '13_99', '13_100', '13_102',
       '13_7', '13_103', '13_104', '14_106', '14_107', '14_108', '14_109',
       '14_110', '14_111', '14_112', '14_114', '14_7', '14_115', '15_116',
       '15_117'

In [9]:
test = pd.read_csv('Mapping.csv')
test.head()

,Unnamed: 0,City,Financial Year,Quarter Ending,Quarter,key
0,0,1,2024,12/31/2023,2,123120231
1,1,1,2024,09/30/2023,1,93020231
2,2,1,2023,06/30/2023,4,63020231
3,3,1,2023,03/31/2023,3,33120231
4,4,1,2023,12/31/2022,2,123120221


In [10]:
test.shape

(3972, 6)

In [11]:
test.isnull().sum()

Unnamed: 0        0
City              0
Financial Year    0
Quarter Ending    0
Quarter           0
key               0
dtype: int64

In [12]:
train['Quarter Ending'] = pd.to_datetime(train['Quarter Ending'])

In [13]:
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_absolute_percentage_error

In [28]:
features = train[['Quarter Ending', 'Quarter', 'City']]

In [29]:
features['Quarter Ending'] = features['Quarter Ending'].map(pd.Timestamp.toordinal)  # Convert dates to ordinal

C:\Users\ahpat\AppData\Local\Temp\ipykernel_11420\2514656071.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features['Quarter Ending'] = features['Quarter Ending'].map(pd.Timestamp.toordinal)  # Convert dates to ordinal


In [30]:
target = train['GDP'].astype(float)

In [31]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.ensemble import RandomForestRegressor

In [32]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42, shuffle=False)

In [33]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [34]:
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train_scaled, y_train)

RandomForestRegressor(random_state=42)

In [35]:
y_pred = model.predict(X_test_scaled)

In [36]:
mape = mean_absolute_percentage_error(y_test, y_pred)
mape

19.70179116510153

In [40]:
test['Quarter Ending'] = pd.to_datetime(test['Quarter Ending'])

In [42]:
test['Quarter Ending'] = test['Quarter Ending'].map(pd.Timestamp.toordinal)  

In [45]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33734 entries, 0 to 33733
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      33734 non-null  int64  
 1   Financial Year  33734 non-null  int64  
 2   Quarter Ending  33734 non-null  int64  
 3   Quarter         33734 non-null  int64  
 4   County          33734 non-null  int64  
 5   City            33734 non-null  int64  
 6   County_City     33734 non-null  object 
 7   key             33734 non-null  int64  
 8   GDP             33734 non-null  float64
dtypes: float64(1), int64(7), object(1)
memory usage: 2.3+ MB


In [48]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3972 entries, 0 to 3971
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   Unnamed: 0      3972 non-null   int64
 1   City            3972 non-null   int64
 2   Financial Year  3972 non-null   int64
 3   Quarter Ending  3972 non-null   int64
 4   Quarter         3972 non-null   int64
 5   key             3972 non-null   int64
dtypes: int64(6)
memory usage: 186.3 KB


In [49]:
x_train = train[['Quarter Ending', 'Quarter', 'City']]
y_train = train['GDP']
x_test = test[['Quarter Ending', 'Quarter', 'City']]

In [53]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(x_train)
X_test_scaled = scaler.transform(x_test)

In [54]:
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train_scaled, y_train)

RandomForestRegressor(random_state=42)

In [55]:
y_pred = model.predict(X_test_scaled)

In [56]:
type(y_pred)

numpy.ndarray

In [57]:
submission = pd.read_csv('Submission.csv')
submission.head()

,Unnamed: 0,GDP
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


In [65]:
solution = pd.DataFrame(submission['Unnamed: 0'])
solution['GDP'] = np.round(y_pred,0)
solution

,Unnamed: 0,GDP
0,0,1620338.0
1,1,1507227.0
2,2,1107641.0
3,3,1479239.0
4,4,1620338.0
...,...,...
3967,3967,1703724.0
3968,3968,1735179.0
3969,3969,1853646.0
3970,3970,1766313.0


In [66]:
solution.to_csv('Solution.csv', index= False)